In [ ]:
from IPython.display import HTML
from IPython.display import display

display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
%matplotlib inline
import numpy as np, scipy, scipy.stats as stats, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import statsmodels, statsmodels.api as sm
import sympy, sympy.stats
import pymc3 as pm
import daft
import xarray, numba, arviz as az

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(linewidth=1000)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

SEED = 42
np.random.seed(SEED)

sns.set()

import warnings
warnings.filterwarnings("ignore")

This blog post is part of the [Series: Monte Carlo Methods](https://weisser-zwerg.dev/posts/series-monte-carlo-methods/).

You can find this blog post on [weisser-zwerg.dev](https://weisser-zwerg.dev/posts/monte-carlo-markov-chain-monte-carlo/) or on [github](https://github.com/cs224/blog-series-monte-carlo-methods) as either [html](https://rawcdn.githack.com/cs224/blog-series-monte-carlo-methods/main/0020-markov-chain-monte-carlo.html) or via [nbviewer](https://nbviewer.jupyter.org/github/cs224/blog-series-monte-carlo-methods/blob/main/0020-markov-chain-monte-carlo.ipynb?flush_cache=true).

# Markov chain Monte Carlo

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Impl" data-toc-modified-id="Impl-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Impl</a></span></li><li><span><a href="#Resources" data-toc-modified-id="Resources-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Resources</a></span></li></ul></div>

## Impl

In [ ]:
# Flip coin 9 times and get 6 heads
samples = np.array([0,0,0,1,1,1,1,1,1])

def fn(a, b):
    return lambda p: stats.bernoulli(p).pmf(samples).prod() * stats.beta(a,b).pdf(p)

# convert from omega, kappa parametrization of the beta distribution to the alpha, beta parametrization
def ok2ab(omega, kappa):
    return omega*(kappa-2)+1,(1-omega)*(kappa-2)+1


In [ ]:
@numba.jit(nopython=True)
def bernoulli(p, samples):
    r = np.zeros_like(samples, dtype=numba.float64)
    for i in range(len(r)):
        if samples[i] < 0.5: # == 0
            r[i] = np.log(1-p)
        else: # == 1
            r[i] = np.log(p)
    return np.sum(r)

In [ ]:
bernoulli(0.001, samples)

In [ ]:
# verify that our implementation delivers the same results as the scipy implementation
stats.bernoulli(0.001).logpmf(samples).sum()

In [ ]:
@numba.jit(nopython=True)
def logpdf(p):
    return bernoulli(p,samples)

@numba.jit(nopython=True)
def mcmc(q_rvs, unif_rvs, trace, logpdf):
    p = 0.5
    for i in range(N):
        rv = q_rvs[i]
        unifrand = unif_rvs[i]
        p_new = p + rv
        log_hastings_ratio = logpdf(p_new) - logpdf(p) # is only correct, because rv is from a symmetric distribution otherwise the "Hastings q(y,x)/q(x,y) is missing"
        if log_hastings_ratio >= 0.0 or unifrand < np.exp(log_hastings_ratio):
            p = p_new
        trace[i] = p
    return trace

In [ ]:
N = 10000

q_rvs = stats.norm(0,0.3).rvs(size=N, random_state=np.random.RandomState(42))
unif_rvs = stats.uniform.rvs(size=N, random_state=np.random.RandomState(42))
trace = np.zeros(N)
mcmc(q_rvs, unif_rvs, trace, logpdf)
trace

In [ ]:
datadict = {'p': trace}
dataset = az.convert_to_inference_data(datadict)
dataset

In [ ]:
az.summary(dataset)

In [ ]:
az.plot_trace(dataset)

In [ ]:
def kdeplot(lds, parameter_name=None, x_min = None, x_max = None, ax=None, kernel='gau'):
    if parameter_name is None and isinstance(lds, pd.Series):
        parameter_name = lds.name
    kde = sm.nonparametric.KDEUnivariate(lds)
    kde.fit(kernel=kernel, fft=False, gridsize=2**10)
    if x_min is None:
        x_min = kde.support.min()
    else:
        x_min = min(kde.support.min(), x_min)
    if x_max is None:
        x_max = kde.support.max()
    else:
        x_max = max(kde.support.max(),  x_max)
    x = np.linspace(x_min, x_max,100)
    y = kde.evaluate(x)
    if ax is None:
        plt.figure(figsize=(6, 3), dpi=80, facecolor='w', edgecolor='k')
        ax = plt.subplot(1, 1, 1)
    ax.plot(x, y, lw=2)
    ax.set_xlabel(parameter_name)
    ax.set_ylabel('Density')

In [ ]:
plt.figure(figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
ax = plt.subplot(1, 1, 1)
kdeplot(trace, x_min=0.0, x_max=1.0, ax=ax)
x = np.linspace(0.0,1.0,100)
y = stats.beta(1+6,1+3).pdf(x)
ax.plot(x,y)

In [ ]:
with pm.Model() as model:
    p = pm.Beta('p', 1.0, 1.0)
    yl = pm.Bernoulli('yl', p, observed=samples)
    prior = pm.sample_prior_predictive()
    posterior = pm.sample(return_inferencedata=True)
    posterior_pred = pm.sample_posterior_predictive(posterior)    

In [ ]:
pm.summary(posterior)

In [ ]:
ldf = pd.DataFrame(datadict)
ldf['w'] = 1.0/len(ldf)
ldf = ldf.sort_values('p').set_index('p')
ldf['c'] = ldf['w'].cumsum()
ldf1 = ldf
ldf1

In [ ]:
ldf = posterior['posterior']['p'].loc[dict(chain=0)].to_dataframe()
ldf = ldf.drop('chain', axis=1)
ldf['w'] = 1.0/len(ldf)
ldf = ldf.sort_values('p').set_index('p')
ldf['c'] = ldf['w'].cumsum()
ldf2 = ldf
ldf2

In [ ]:
plt.figure(figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
ax = plt.subplot(1, 1, 1)
x = np.linspace(0.0,1.0,100)
y = stats.beta(1+6,1+3).cdf(x)
ax.plot(x,y, label='exact')
ldf1.loc[0.0:1.0, 'c'].plot(ax=ax, label='self-made MCMC')
ldf2.loc[0.0:1.0, 'c'].plot(ax=ax, label='PyMC3')
ax.legend()

## Resources

Text books:

* [Handbook of Markov Chain Monte Carlo](https://www.amazon.com/-/de/dp-B008GXJVF8/dp/B008GXJVF8/) by Steve Brooks, Andrew Gelman, Galin Jones, Xiao-Li Meng
* [Machine Learning: A Bayesian and Optimization Perspective](https://www.amazon.com/-/de/dp/0128188030) by [Sergios Theodoridis](https://sergiostheodoridis.wordpress.com/)
    * Chapter 14.7 Markov Chain Monte Carlo Methods
* [Probabilistic Graphical Models: Principles and Techniques](https://www.amazon.com/Probabilistic-Graphical-Models-Principles-Computation/dp/0262013193) by Daphne Koller and Nir Friedman
    * Chapter 12.3 Markov Chain Monkte Carlo Methods
* [Bayesian Reasoning and Machine Learning](https://www.amazon.com/Bayesian-Reasoning-Machine-Learning-Barber/dp/0521518148) by David Barber
    * Chapter 27.4 Markov chain Monte Carlo (MCMC)
* [Pattern Recognition and Machine Learning](https://www.amazon.com/Pattern-Recognition-Learning-Information-Statistics/dp/1493938436) by Christopher M. Bishop
    * Chapter 11.2 Markov Chain Monte Carlo
* [Machine Learning: A Probabilistic Perspective](https://www.amazon.com/Machine-Learning-Probabilistic-Perspective-Computation/dp/0262018020/)
    * Chapter 24 Markov chain Monte Carlo (MCMC) inference
* [Doing Bayesian Data Analysis](https://www.amazon.com/-/de/dp/0124058884/): A Tutorial with R, JAGS, and Stan by [John Kruschke](http://doingbayesiandataanalysis.blogspot.com/)
    * Chapter 7 Markov Chain Monte Carlo
* [Statistical Rethinking](https://www.amazon.com/-/de/dp/036713991X): A Bayesian Course with Examples in R and STAN by [Richard McElreath](https://elevanth.org/blog/)
    * Chapter 9 Markov Chain Monte Carlo

Tutorial:

* [A Tutorial on Markov Chain Monte-Carlo and Bayesian Modeling](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3759243) by Martin B. Haugh


Blog posts:

* [Series of posts on implementing Hamiltonian Monte Carlo](https://discourse.pymc.io/t/series-of-posts-on-implementing-hamiltonian-monte-carlo/3138) by Colin Carroll
    * [Hamiltonian Monte Carlo from scratch](https://colindcarroll.com/2019/04/11/hamiltonian-monte-carlo-from-scratch/)
    * [Step Size Adaptation in Hamiltonian Monte Carlo](https://colindcarroll.com/2019/04/21/step-size-adaptation-in-hamiltonian-monte-carlo/)
    * [Choice of Symplectic Integrator in Hamiltonian Monte Carlo](https://colindcarroll.com/2019/04/28/choice-of-symplectic-integrator-in-hamiltonian-monte-carlo/)
    * [Pragmatic Probabilistic Programming](https://colcarroll.github.io/hmc_tuning_talk/)
    * [A tour of probabilistic programming language APIs](https://colcarroll.github.io/ppl-api/)
    * [minimc](https://github.com/ColCarroll/minimc) ~15 line Hamiltonian Monte Carlo implementation
    * [Hamiltonian Monte Carlo in PyMC3](https://colcarroll.github.io/hamiltonian_monte_carlo_talk/bayes_talk.html)
* [Markov Chains: Why Walk When You Can Flow?](https://elevanth.org/blog/2017/11/28/build-a-better-markov-chain/) by Richard McElreath
* [Bayesian Inference Algorithms: MCMC and VI](https://towardsdatascience.com/bayesian-inference-algorithms-mcmc-and-vi-a8dad51ad5f5) by Wicaksono Wijono

Papers:

* [Mixed Hamiltonian Monte Carlo for Mixed Discrete and Continuous Variables](https://arxiv.org/abs/1909.04852) by Guangyao Zhou
    * [Guangyao (Stannis) Zhou](https://stanniszhou.github.io/)
    * [mixed_hmc](https://github.com/StannisZhou/mixed_hmc) on GitHub
    * [MixedHMC](http://num.pyro.ai/en/latest/mcmc.html#numpyro.infer.mixed_hmc.MixedHMC) for NumPyro
    * [YouTube](https://www.youtube.com/watch?v=ag44SuB0wB8)